In [10]:
view(Atoms('F'))

In [2]:
from dependencies import * 

_ColormakerRegistry()

# Data read

In [11]:
outdir = "../cf4_bomb1_outputs/"
indir = "../cf4_bomb1_inputs/"

## Structures

In [12]:
def readData(outdir, indir, useRemoval = True, useFrags = True, useBonds = True, wrapStructs = True):
    """
    Read structures; append removal data from cluster, fragment data, and bond data optionally
    Wrap structures optionally
    """
    
    # read structures, initialize df
    
    data = pd.concat([readStructs(indir, shallow = False, name = 'input'),
                         readStructs(outdir, shallow = False)
                        ], axis = 1)
    data.columns = ['in', 'out']
    for i in data['in']:
        i.wrap()
    
    if useRemoval:  
        bombdata, quenchdata, eqdata = postprocessResults(outdir)

        newdata = {}
        for idx in data.index:
            i, j = idx.split("-") # for i-j type output
            i = int(i)
            newdata[idx] = data.loc[idx].append(bombdata[i][j])
        data = pd.DataFrame(newdata).T
        ncols = data.shape[1]
        data.columns = pd.MultiIndex.from_arrays(
            [["struct"] * 2 + ["removal"] * (ncols - 2), data.columns],
            names = ['source', 'data']
        )
            
    
    if useFrags:
        infrags = pd.read_csv(indir+"fragdata.csv", index_col=0)
        outfrags = pd.read_csv(outdir+"fragdata.csv", index_col=0)

        fragdiffs = outfrags.subtract(infrags, fill_value = 0)
        ncols = fragdiffs.shape[1]
        fragdiffs.columns = pd.MultiIndex.from_arrays(
            [['frags'] * ncols, fragdiffs.columns], names = ['source', 'data']
        )

        data = pd.concat([data, fragdiffs], axis = 1)
    

    if useBonds:
        outbonds = pd.read_csv(outdir + "bondcounts.csv", index_col=0)

        inbonds = pd.read_csv(indir + "bondcounts.csv", index_col=0)

        bondDiffs = outbonds.subtract(inbonds)
        
        ncols = bondDiffs.shape[1]
        bondDiffs.columns = pd.MultiIndex.from_arrays(
            [['bonds'] * ncols, bondDiffs.columns], names = ['source', 'data']
        )

        data = pd.concat([data, bondDiffs], axis = 1)
    return data

In [13]:
data = readData(outdir, indir, useRemoval=True, useFrags=True, useBonds=True)
data.head()

source                                             struct  \
data                                                   in   
4-5     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
5-16    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
4-10    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
3-11    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
2-4     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   

source                                                    removal              \
data                                                  out      Si  N  H Ar  F   
4-5     (Atom('H', [5.687816897, 11.8095906, 0.4501787...       0  0  1  1  4   
5-16    (Atom('H', [5.687816897, 11.8095906, 0.4501787...       0  0  0  1  1   
4-10    (Atom('H', [5.687816897, 11.8095906, 0.4501787...       0  0  0  1  2   
3-11    (Atom('H', [5.687816897, 11.8095906, 0.4501787...       0  0  0  1  2   
2-4     (Atom('H', [5.687816897, 11.8095906, 0.4501787...       0  0  0  0  0   

source    frags     ... bonds                                          
data    C    Ar  C  ...   F-N F-C F-H F-Ar N-C N-H N-Ar C-H C-Ar H-Ar  
4-5     1     0  0  ...     0   8   0    0   0  -1    0   0    0    0  
5-16    1     0  0  ...     0   8   0    0   0   0    0   0    0    0  
4-10    0     1  0  ...     0   0   0    0   0   0    0   0    0    0  
3-11    0     0  0  ...     0   8   0    0   0   0    0   0    0    0  
2-4     0     0  0  ...     0   4   0    0   0   1    0   0    0    0  

[5 rows x 45 columns]

In [149]:
data['frags']

data,Ar,C,C2,C2H,CF,CF2,CF3,CF4,CH2Si,CHN,...,CSi,F,H,H2N,H2N2Si,H3N,HN,HNSi,N,NSi
4-5,-1,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,-2,0,0.0,0,0,0,0.0,0,0
5-16,0,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,-1,0.0,0,0
4-10,-1,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,-1,0,0.0,0,0,0,0.0,0,0
3-11,-1,0.0,0.0,0.0,0,0,0,-2,0.0,0,...,0,5,0,0.0,0,0,0,0.0,0,0
2-4,-1,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,-1,0,0.0,0,0,1,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1-10,-1,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,-3,0,0.0,0,0,1,0.0,0,0
0-1,0,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0
1-9,-2,0.0,0.0,0.0,0,0,-1,-1,0.0,0,...,0,2,-1,0.0,0,0,0,0.0,0,0
4-7,-1,0.0,0.0,0.0,0,0,0,0,0.0,0,...,0,-1,0,0.0,0,0,0,0.0,0,0


# Fragment formation

In [145]:
sinDiffCols = [col[0] == 'frags' and 
              np.any(
                  np.isin(
                      list(Formula(col[1]).count().keys()), ["Si", "N"])) and 
              np.any(
                  np.isin(
                      list(Formula(col[1]).count().keys()), ["C", "F"])) 
 for col in data.columns]

In [148]:
idx = pd.IndexSlice
sinDiffs = data.loc[:, np.logical_or(sinDiffCols, ["struct" in i for i in data.columns])]
sinDiffs = sinDiffs[[sum(i) != 0 for _, i in sinDiffs['frags'].iterrows()]]
sinDiffs

source                                             struct  \
data                                                   in   
4-6     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
1-0     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
3-3     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
2-15    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
9-8     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
9-12    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
9-2     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
5-9     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
4-16    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
6-8     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
8-13    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   

source                                                    frags             
data                                                  out CH2Si CHN CN CSi  
4-6     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  1   0  
1-0     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  0  -1  
3-3     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  0  -1  
2-15    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  1   0  
9-8     (Atom('H', [5.687816897, 11.8095906, 0.4501787...  -1.0   0  0   0  
9-12    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  0  -1  
9-2     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0 -1   0  
5-9     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  1   0  
4-16    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0 -1   0  
6-8     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0 -1   0  
8-13    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  0   1

In [143]:
idx = pd.IndexSlice
hasSiN = data.loc[:, np.logical_or(hasSiNCols, ["struct" in i for i in data.columns])]
hasSiN = hasSiN[[sum(i) < 0 for _, i in hasSiN['frags'].iterrows()]]
hasSiN

source                                             struct  \
data                                                   in   
1-0     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
3-3     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
9-8     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
9-12    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
9-2     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
4-16    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
6-8     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   

source                                                    frags             
data                                                  out CH2Si CHN CN CSi  
1-0     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  0  -1  
3-3     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  0  -1  
9-8     (Atom('H', [5.687816897, 11.8095906, 0.4501787...  -1.0   0  0   0  
9-12    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0  0  -1  
9-2     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0 -1   0  
4-16    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0 -1   0  
6-8     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   0.0   0 -1   0

In [142]:
sum(data.loc[:, ('frags', 'CH2Si')])

-1.0

# Removal

In [151]:
si_removal = data[data['removal']["Si"] > 0]
si_removal

source                                             struct  \
data                                                   in   
3-3     (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
9-12    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   
5-13    (Atom('H', [5.687816897, 11.8095906, 0.4501787...   

source                                                    removal              \
data                                                  out      Si  N  H Ar  F   
3-3     (Atom('H', [5.687816897, 11.8095906, 0.4501787...       1  0  1  0  0   
9-12    (Atom('H', [5.687816897, 11.8095906, 0.4501787...       1  1  1  0  0   
5-13    (Atom('H', [5.687816897, 11.8095906, 0.4501787...       1  2  1  0  3   

source    frags       ... bonds                                          
data    C    Ar    C  ...   F-N F-C F-H F-Ar N-C N-H N-Ar C-H C-Ar H-Ar  
3-3     1    -1  0.0  ...     0   0   0    0   0   0    0   0    0    0  
9-12    1    -1  0.0  ...     0  -4   0    0   0  -1    0   0    0    0  
5-13    0    -1  0.0  ...     0  -1   0    0   0  -1    0   0    0    0  

[3 rows x 44 columns]

In [153]:
view(si_removal.loc['3-3', ('struct',['in', 'out'])])

In [8]:
view(si_removal['in'])

# Si-F Bonds

In [35]:
totalData[totalData['Si-F'] > 0]

,in,out,Si,N,H,Ar,F,C,Si-F,Si-N,...,F-N,F-C,F-H,F-Ar,N-C,N-H,N-Ar,C-H,C-Ar,H-Ar
4-4,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...",0,0,0,0,0,0,1,-5,...,0,0,0,0,0,0,0,0,0,0
0-2,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...",0,0,0,1,5,0,2,-3,...,0,-8,0,0,0,0,0,0,0,0
1-3,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...",0,0,0,0,0,0,1,-3,...,0,0,0,0,0,0,0,0,0,0
0-0,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...",0,0,0,1,2,0,1,-4,...,0,-4,0,0,2,0,0,0,0,0
1-6,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...",0,0,0,0,1,0,1,3,...,0,0,0,0,0,0,0,0,0,0
0-14,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...",0,0,1,1,0,0,1,-2,...,0,0,0,0,0,0,0,0,0,0
2-12,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...",0,2,3,1,1,0,1,-5,...,0,-3,0,0,0,-3,0,1,0,0


In [36]:
view(totalData.loc[totalData['Si-F'] > 0, 'in'])